In [1]:
import sys
from pathlib import Path
import mlflow

# 1. Aggiungi la root del progetto al path di Python
#    Path.cwd() è la directory del notebook: '.../notebooks'
#    .parent è la root del progetto: '.../radio-data-curation-ssl'
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# 2. Imposta ESPLICITAMENTE il Tracking URI di MLflow
mlflow_tracking_uri = project_root / "mlruns"
mlflow.set_tracking_uri(mlflow_tracking_uri.as_uri())

print(f"Project root impostato su: {project_root}")
print(f"MLflow tracking URI impostato su: {mlflow.get_tracking_uri()}")


from pathlib import Path
from src.data.data_loaders import (
    CutoutLoader,
    FeaturesLoader,
    ClusteringLoader,
    SamplingLoader,
    SourceNotFoundError,
    MLflowArtifactLoader
)

Project root impostato su: /leonardo_work/INA24_C5B09/radio-data-curation-ssl
MLflow tracking URI impostato su: file:///leonardo_work/INA24_C5B09/radio-data-curation-ssl/mlruns


In [2]:
# =============================================================================
# SNIPPET 1: Caricare e Ispezionare una Lista di Cutout
# =============================================================================
print("--- Ispezione Artefatto: Cutout List ---")
# Prerequisito: Trova il path di un file info.json generato da cutout.py o merge_cutouts.py
#path_to_cutout_list = Path("outputs/merged_cutouts/merged_cutouts_...hash.../info.json")
path_to_cutout_list = Path("/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/cutouts/cutouts-strategy_sliding_window-overlap_05-size_256_30d67552/info.json")

try:
    cutout_data = CutoutLoader.from_json(path_to_cutout_list)
    
    print(f"Caricamento riuscito da: {path_to_cutout_list}")
    print(f"Numero totale di cutout trovati: {len(cutout_data.cutouts)}")
    
    # Ispeziona il primo cutout per verificarne la struttura
    if cutout_data.cutouts:
        first_cutout = cutout_data.cutouts[0]
        print("\nDettagli del primo cutout:")
        print(f"  ID: {first_cutout.id}")
        print(f"  Path: {first_cutout.file_path}")
        print(f"  Survey: {first_cutout.survey}")
        print(f"  Posizione: {first_cutout.position}")

except SourceNotFoundError as e:
    print(f"ERRORE: Impossibile caricare la lista dei cutout. {e}")
except Exception as e:
    print(f"ERRORE INATTESO: {e}")

--- Ispezione Artefatto: Cutout List ---
Caricamento riuscito da: /leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/cutouts/cutouts-strategy_sliding_window-overlap_05-size_256_30d67552/info.json
Numero totale di cutout trovati: 3112435

Dettagli del primo cutout:
  ID: 0
  Path: outputs/cutouts/cutouts-strategy_sliding_window-overlap_05-size_256_30d67552/P184+57/npy/patch_128_3072.npy
  Survey: LoTTS
  Posizione: (128, 3072)


In [9]:
# =============================================================================
# SNIPPET 2: Caricare Features e Percorsi Immagini dall'Archivio HDF5
# =============================================================================
print("\n--- Ispezione Artefatto: Features & Image Paths Archive ---")
# Prerequisito: Trova il path di un file .h5 generato da feature_extraction.py
path_to_features_h5 = Path("/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/features/features-model_vit_small_patch14_reg4_dinov2lvd142m-variant__b27e541a/features_data_fast.h5")
path_to_features_npy = Path("/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/features/features-model_vit_small_patch14_reg4_dinov2lvd142m-variant__64effbed/features.npy")
mlflow_id = "94511e7430364c938bdaf44d725dc36c"

# 2a: Caricare solo le features
try:
    features_data = FeaturesLoader.load_features_from_path(path_to_features_h5)
    
    print(f"Caricamento features riuscito da: {path_to_features_h5}")
    print(f"Shape dell'array di features: {features_data.features.shape}")
    print(f"  (Numero di campioni, Dimensione delle features)")
    
except SourceNotFoundError as e:
    print(f"ERRORE: Impossibile caricare le features. {e}")
except Exception as e:
    print(f"ERRORE INATTESO: {e}")

# 2b: Caricare solo i percorsi delle immagini
try:
    image_paths_data = FeaturesLoader.load_image_paths_from_h5(path_to_features_h5)
    
    print(f"\nCaricamento percorsi immagini riuscito da: {path_to_features_h5}")
    print(f"Numero totale di percorsi trovati: {len(image_paths_data.paths)}")

    if image_paths_data.paths:
        print("\nPrimi 3 percorsi trovati:")
        for p in image_paths_data.paths[:3]:
            print(f"  - {p}")
            
except SourceNotFoundError as e:
    print(f"ERRORE: Impossibile caricare i percorsi delle immagini. {e}")
except Exception as e:
    print(f"ERRORE INATTESO: {e}")


--- Ispezione Artefatto: Features & Image Paths Archive ---
Caricamento features riuscito da: /leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/features/features-model_vit_small_patch14_reg4_dinov2lvd142m-variant__b27e541a/features_data_fast.h5
Shape dell'array di features: (9606, 384)
  (Numero di campioni, Dimensione delle features)

Caricamento percorsi immagini riuscito da: /leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/features/features-model_vit_small_patch14_reg4_dinov2lvd142m-variant__b27e541a/features_data_fast.h5
Numero totale di percorsi trovati: 9606

Primi 3 percorsi trovati:
  - /leonardo_work/INA24_C5B09/lofar_rgz_dataset/lotss_dr2_morph_horton-dataset/data/npy/ILTJ000003.76+314705.5.npy
  - /leonardo_work/INA24_C5B09/lofar_rgz_dataset/lotss_dr2_morph_horton-dataset/data/npy/ILTJ000005.31+205913.2.npy
  - /leonardo_work/INA24_C5B09/lofar_rgz_dataset/lotss_dr2_morph_horton-dataset/data/npy/ILTJ000025.69+342004.8.npy


In [10]:
# =============================================================================
# SNIPPET 2: Caricare Features e Percorsi Immagini dal npy RETROCOMPATIBLITA
# =============================================================================
print("\n--- Ispezione Artefatto: Features & Image Paths Archive ---")
# Prerequisito: Trova il path di un file .npy generato da feature_extraction.py
path_to_features_npy = Path("/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/features/features-model_vit_small_patch14_reg4_dinov2lvd142m-variant__64effbed/features.npy")
mlflow_id = "94511e7430364c938bdaf44d725dc36c"

# 2a: Caricare solo le features
try:
    features_data = FeaturesLoader.load_features_from_path(path_to_features_npy)
    
    print(f"Caricamento features riuscito da: {path_to_features_npy}")
    print(f"Shape dell'array di features: {features_data.features.shape}")
    print(f"  (Numero di campioni, Dimensione delle features)")
    
except SourceNotFoundError as e:
    print(f"ERRORE: Impossibile caricare le features. {e}")
except Exception as e:
    print(f"ERRORE INATTESO: {e}")
           
except SourceNotFoundError as e:
    print(f"ERRORE: Impossibile caricare i percorsi delle immagini. {e}")
except Exception as e:
    print(f"ERRORE INATTESO: {e}")


--- Ispezione Artefatto: Features & Image Paths Archive ---
Caricamento features riuscito da: /leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/features/features-model_vit_small_patch14_reg4_dinov2lvd142m-variant__64effbed/features.npy
Shape dell'array di features: (3112435, 384)
  (Numero di campioni, Dimensione delle features)


In [7]:
# =============================================================================
# SNIPPET 3: Caricare i Risultati del Clustering
# =============================================================================
print("\n--- Ispezione Artefatto: Clustering Results ---")
# Prerequisito: Trova il path della DIRECTORY di output di un'esecuzione di clustering specificando
#path_to_clustering_dir = Path("path/to/your/clustering_exp_dir/")
path_to_clustering_dir = Path("/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/clusters/clusters-n_levels_2-level0_5000-level1_50_a33db3d6/level2")


try:
    clustering_data = ClusteringLoader.from_directory(path_to_clustering_dir)
    
    print(f"Caricamento risultati clustering riuscito da: {path_to_clustering_dir}")
    print(f"Numero di cluster trovati: {len(clustering_data.clusters)}")
    print(f"Shape dell'array dei centroidi: {clustering_data.centroids.shape}")
    
    if clustering_data.clusters:
        print(f"\nNumero di elementi nel primo cluster: {len(clustering_data.clusters[0])}")
        print(f"Primi 5 indici del primo cluster: {clustering_data.clusters[0][:5]}")

except SourceNotFoundError as e:
    print(f"ERRORE: Impossibile caricare i risultati del clustering. {e}")
except Exception as e:
    print(f"ERRORE INATTESO: {e}")



--- Ispezione Artefatto: Clustering Results ---
Caricamento risultati clustering riuscito da: /leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/clusters/clusters-n_levels_2-level0_5000-level1_50_a33db3d6/level2
Numero di cluster trovati: 50
Shape dell'array dei centroidi: (50, 512)

Numero di elementi nel primo cluster: 14
Primi 5 indici del primo cluster: [ 55 394 122 204 229]


In [4]:
# Aggiungi a inspect_artifacts.py o al notebook

# =============================================================================
# SNIPPET 6: Caricare una Gerarchia di Clustering Complessa via MLflow
# =============================================================================
print("\n--- Ispezione Artefatto: Gerarchia di Clustering ---")
path_to_clustering_dir = Path("/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/clusters/clusters-n_levels_2-level0_5000-level1_50_a33db3d6")

try:
    # ID della run di 'clustering'
    clustering_data = ClusteringLoader.from_directory(path_to_clustering_dir)
    
    print(f"OK - Caricata gerarchia di clustering dal path '{path_to_clustering_dir}'")
    print(f"   Numero totale di livelli: {clustering_data.n_levels}")
    
    for level in clustering_data.levels:
        print(f"   - Livello {level.level_index}:")
        print(f"     > Numero di cluster: {len(level.clusters)}")
        print(f"     > Shape centroidi: {level.centroids.shape}")

except SourceNotFoundError as e:
    print(f"ERRORE nel caricare il clustering dal path '{path_to_clustering_dir}': {e}")


--- Ispezione Artefatto: Gerarchia di Clustering ---
OK - Caricata gerarchia di clustering dal path '/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/clusters/clusters-n_levels_2-level0_5000-level1_50_a33db3d6'
   Numero totale di livelli: 2
   - Livello 0:
     > Numero di cluster: 5000
     > Shape centroidi: (5000, 512)
   - Livello 1:
     > Numero di cluster: 50
     > Shape centroidi: (50, 512)


In [2]:
# Aggiungi a inspect_artifacts.py o al notebook

# =============================================================================
# SNIPPET 6: Caricare una Gerarchia di Clustering Complessa via MLflow
# =============================================================================
print("\n--- Ispezione Artefatto: Gerarchia di Clustering via MLflow ---")

try:
    # ID della run di 'clustering'
    clustering_run_id = "38c2ef82a74c4b948af8dd7d5f14741c" 
    
    clustering_data = MLflowArtifactLoader.load_clustering_from_mlflow(
        clustering_run_id, 
        project_root=project_root
    )
    
    print(f"OK - Caricata gerarchia di clustering dalla run '{clustering_run_id}'")
    print(f"   Numero totale di livelli: {clustering_data.n_levels}")
    
    for level in clustering_data.levels:
        print(f"   - Livello {level.level_index}:")
        print(f"     > Numero di cluster: {len(level.clusters)}")
        print(f"     > Shape centroidi: {level.centroids.shape}")

except SourceNotFoundError as e:
    print(f"ERRORE nel caricare il clustering dalla run '{clustering_run_id}': {e}")


--- Ispezione Artefatto: Gerarchia di Clustering via MLflow ---
OK - Caricata gerarchia di clustering dalla run '38c2ef82a74c4b948af8dd7d5f14741c'
   Numero totale di livelli: 2
   - Livello 0:
     > Numero di cluster: 5000
     > Shape centroidi: (5000, 512)
   - Livello 1:
     > Numero di cluster: 50
     > Shape centroidi: (50, 512)


In [6]:
# =============================================================================
# SNIPPET 4: Caricare gli Indici Campionati
# =============================================================================
print("\n--- Ispezione Artefatto: Sampled Indices ---")
# Prerequisito: Trova il path di un file .npy generato da sampling.py
path_to_sampled_indices = Path("/leonardo_work/INA24_C5B09/radio-data-curation-ssl/outputs/clusters/clusters-n_levels_2-level0_5000-level1_50_a33db3d6/subsampled_indices-target_size_30000_fe1e6f21.npy")

try:
    sampled_indices_data = SamplingLoader.from_npy(path_to_sampled_indices)
    
    print(f"Caricamento indici riuscito da: {path_to_sampled_indices}")
    print(f"Numero totale di indici campionati: {len(sampled_indices_data.indices)}")
    
    if sampled_indices_data.indices.size > 0:
        print(f"Valore minimo/massimo degli indici: {sampled_indices_data.indices.min()} / {sampled_indices_data.indices.max()}")
        print(f"Primi 10 indici campionati: {sampled_indices_data.indices[:10]}")

except SourceNotFoundError as e:
    print(f"ERRORE: Impossibile caricare gli indici campionati. {e}")
except Exception as e:
    print(f"ERRORE INATTESO: {e}")


--- Ispezione Artefatto: Sampled Indices ---
ERRORE: Impossibile caricare gli indici campionati. File di indici campionati non trovato: path/to/your/clustering_exp_dir/curated_datasets/1c_mul1_..._selection.npy


In [3]:
# Importa la nuova classe facade
# =============================================================================
# SNIPPET 5: Caricare artefatti tramite MLflow Run ID
# =============================================================================
print("\n--- Ispezione Artefatti via MLflow ---")

# --- 5a: Caricare una MergedCutoutList ---
try:
    # ID della run di 'merge_cutouts'
    merge_run_id = "c6e73956a66a49f98117157279342b47" 
    merged_cutouts = MLflowArtifactLoader.load_cutout_list_from_mlflow(merge_run_id, project_root=project_root)
    print(f"OK - Caricata MergedCutoutList dalla run '{merge_run_id}'")
    print(f"   Trovati {len(merged_cutouts.cutouts)} cutout.")
except SourceNotFoundError as e:
    print(f"ERRORE nel caricare la MergedCutoutList dalla run '{merge_run_id}': {e}")


--- Ispezione Artefatti via MLflow ---
OK - Caricata MergedCutoutList dalla run 'c6e73956a66a49f98117157279342b47'
   Trovati 3112435 cutout.


In [2]:
try:
    features_run_id = "94511e7430364c938bdaf44d725dc36c"
    # Passa `project_root` al metodo!
    features_data = MLflowArtifactLoader.load_features_from_mlflow(features_run_id, project_root=project_root)
    
    print(f"\nOK - Caricate Features (compatibile .npy) dalla run '{features_run_id}'")
    print(f"   Shape delle features: {features_data.features.shape}")
except SourceNotFoundError as e:
    print(f"ERRORE nel caricare le Features dalla run '{features_run_id}': {e}")


OK - Caricate Features (compatibile .npy) dalla run '94511e7430364c938bdaf44d725dc36c'
   Shape delle features: (3112435, 384)
